In [1]:
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm_notebook

import pandas as pd
import nltk

import config
import log

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/colan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Loading in Data

In [3]:
log.info("Loading Training Data")

# use a low n-rows when testing else don't set it
# df = pd.read_csv(config.TRAIN_PATH, header=0, nrows=1000, usecols=['target', 'comment_text'])
df = pd.read_csv(config.TRAIN_PATH, header=0, usecols=['target', 'comment_text'])

INFO: Loading Training Data


In [4]:
df

,target,comment_text
0,0.000000,"This is so cool. It's like, 'would you want yo..."
1,0.000000,Thank you!! This would make my life a lot less...
2,0.000000,This is such an urgent design problem; kudos t...
3,0.000000,Is this something I'll be able to install on m...
4,0.893617,haha you guys are a bunch of losers.
...,...,...
1804869,0.000000,"Maybe the tax on ""things"" would be collected w..."
1804870,0.000000,What do you call people who STILL think the di...
1804871,0.000000,"thank you ,,,right or wrong,,, i am following ..."
1804872,0.621212,Anyone who is quoted as having the following e...


### Simplifying Targets

For this sample we are going to only use the input text and have it guess a sample label. To get the sample label we are going to create 3 intermediaries for the calculated toxicity target betwee 0 and 1.

* 0.00-0.20 -> not toxic
* 0.21-0.60 -> mildly toxic
* 0.61-1.00 -> toxic

In [5]:
targets = df['target'].apply(lambda t: 'not_toxic' if t <= 0.2 else 'mildly_toxic' if t <= 0.6 else 'toxic')
df.update(targets)

In [6]:
df.target.value_counts()

not_toxic       1534680
mildly_toxic     197959
toxic             72235
Name: target, dtype: int64

### Removing Case

In [7]:
targets = df['comment_text'].apply(lambda comment: comment.lower())
df.update(targets)

In [8]:
df

,target,comment_text
0,not_toxic,"this is so cool. it's like, 'would you want yo..."
1,not_toxic,thank you!! this would make my life a lot less...
2,not_toxic,this is such an urgent design problem; kudos t...
3,not_toxic,is this something i'll be able to install on m...
4,toxic,haha you guys are a bunch of losers.
...,...,...
1804869,not_toxic,"maybe the tax on ""things"" would be collected w..."
1804870,not_toxic,what do you call people who still think the di...
1804871,not_toxic,"thank you ,,,right or wrong,,, i am following ..."
1804872,toxic,anyone who is quoted as having the following e...


### Splitting

We split the data so we can estimate the performace of our algorithm. You can set the switch value below to decide the split. Default is to 0.8 where 80% of the training is trained on and 20% is tested on.

In [9]:
split_percentage = 0.8

In [10]:
training_set, testing_set = train_test_split(df, test_size = 1.0 - split_percentage)

log.info(f'Train Data Size: {len(training_set)}')
log.info(f'Test Data Size: {len(testing_set)}')

INFO: Train Data Size: 1443899
INFO: Test Data Size: 360975


### Formatting and Bag of Words

We'll use the 1000 most occurring words for our bag of words in this example.

In [11]:
training_data = []
training_data = []

word_counts = Counter()

In [12]:
train = training_set.to_dict()
test = testing_set.to_dict()

In [13]:
tweet_tokenizer = nltk.tokenize.TweetTokenizer()

for key in tqdm_notebook(train['comment_text']):
    tokens = tweet_tokenizer.tokenize(train['comment_text'][key])
    train['comment_text'][key] = tokens
    
    c = Counter(tokens)
    word_counts.update(c)

In [14]:
most_common_words = word_counts.most_common(1000)

In [15]:
log.info('training set updates')

for key in tqdm_notebook(train['comment_text']):
    tokenized_sentence = train['comment_text'][key]
    bow = {}
    
    for word in most_common_words:
        word = word[0]
        
        if word in tokenized_sentence:
            bow[word] = tokenized_sentence.count(word)
        else:
            bow[word] = 0
            
    train['comment_text'][key] = bow

INFO: training set updates


In [16]:
log.info('testing set updates')

for key in tqdm_notebook(test['comment_text']):
    tokenized_sentence = tweet_tokenizer.tokenize(test['comment_text'][key])
    bow = {}
    
    for word in most_common_words:
        word = word[0]
        
        if word in tokenized_sentence:
            bow[word] = tokenized_sentence.count(word)
        else:
            bow[word] = 0

    test['comment_text'][key] = bow

INFO: testing set updates


In [17]:
log.info('formatting training data for NLTK')

final_training_set = []
for key in tqdm_notebook(train['target']):
    final_training_set.append((train['comment_text'][key], train['target'][key]))

INFO: formatting training data for NLTK


In [18]:
log.info('formatting testing data for nltk')

final_testing_set = []
for key in tqdm_notebook(test['target']):
    final_testing_set.append((test['comment_text'][key], test['target'][key]))

INFO: formatting testing data for nltk


### Training

In [19]:
naive_bayes_classifier = nltk.NaiveBayesClassifier.train(final_training_set)

### Evaluation

In [20]:
print(f'Accuracy: {nltk.classify.accuracy(naive_bayes_classifier, final_testing_set)}')
print(f'Precision: not implemented')
print(f'Recall: not implemented')

0.5117002562504328

In [22]:
naive_bayes_classifier.show_most_informative_features(100)

Most Informative Features
                  stupid = 3               toxic : not_to =    710.7 : 1.0
                  stupid = 4               toxic : not_to =    472.8 : 1.0
                  stupid = 2               toxic : not_to =    415.6 : 1.0
                  stupid = 1               toxic : not_to =    131.4 : 1.0
                  donald = 6               toxic : not_to =     49.4 : 1.0
                       ! = 19              toxic : not_to =     49.4 : 1.0
                       ! = 25              toxic : not_to =     35.3 : 1.0
                       , = 53              toxic : not_to =     35.3 : 1.0
                       ! = 16              toxic : not_to =     30.9 : 1.0
                   white = 10             mildly : not_to =     23.3 : 1.0
                   their = 11             mildly : not_to =     23.3 : 1.0
                   share = 5               toxic : not_to =     21.2 : 1.0
                    play = 6               toxic : not_to =     21.2 : 1.0